In [1]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')

In [2]:
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [11]:
fish_input = fish[['Weight', 'Length', "Diagonal", "Height", "Width"]].to_numpy()
fish_target = fish["Species"].to_numpy()

In [12]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

In [13]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [15]:
from sklearn.linear_model import LogisticRegression

#로지스틱회귀는 max_iter를 통해서 반복횟수를 지정하며 기본값은 100이다
#로지스틱회귀는 L2 규제를 사용하며 매개변수 C가 커질수록 그 규제가 완화된다
lr = LogisticRegression(C=20, max_iter=1000)

lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))


0.9327731092436975
0.925


In [16]:
#각각의 영역으로 분류될 확률을 구해보자
#predict_proba 함수로 출력되는 확률은 아래 classes_ 함수로 구한 순서로 출력된다
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [20]:
import numpy as np
proba = lr.predict_proba(test_scaled[:5])

print(np.round(proba, decimals=3))

[[0.    0.014 0.842 0.    0.135 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.934 0.015 0.016 0.   ]
 [0.011 0.034 0.305 0.006 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


In [23]:
#다중분류는 이진분류와는 다르게 클래스마다 z값을 하나씩 계산한다.
#coef_ 1행과 intercept 1행이 하나의 클래스의 z값을 구하는 계수가 된다.
print(lr.coef_.shape, lr.intercept_.shape)

(7, 5) (7,)


In [25]:
#z값은 decision_function()으로 구할 수 있다.
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=3))

[[ -6.507   1.039   5.168  -2.757   3.337   0.346  -0.625]
 [-10.879   1.944   4.776  -2.423   2.987   7.842  -4.247]
 [ -4.338  -6.237   3.167   6.481   2.364   2.434  -3.872]
 [ -0.691   0.446   2.643  -1.215   3.261  -5.701   1.257]
 [ -6.401  -1.991   5.815  -0.135   3.499  -0.088  -0.699]]


In [29]:
#위에서 구한 z값을 softmax 함수에 대입하면 각 클래스에 대한 확률을 구할 수 있다. 하나의 세트에 대한 모든 클래스의 합은 항상 1이다.
#이진분류에서는 시그모이드 함수를 많이쓰고 다중 분류에서는 softmax함수를 많이 사용한다.
from scipy.special import softmax
proba = softmax(decision, axis=1)
print(np.round(proba, decimals=3))

[[0.    0.014 0.842 0.    0.135 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.934 0.015 0.016 0.   ]
 [0.011 0.034 0.305 0.006 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


In [30]:
#로지스틱 회귀의 predict 함수는 가장 높은 확률을 가진 영역을 출력한다
#classes_ 함수로 찾은 순서대로 확률을 표기한 것이므로 위의 결과와 아래 결과를 비교하면 같은 결과임을 알 수 있다.
lr.predict(test_scaled[:5])

array(['Perch', 'Smelt', 'Pike', 'Roach', 'Perch'], dtype=object)